## Visualization
- Get timestamps from the chosen clips
- Get fMRI from the chosen clips
- Make a file that works for a 3D env
- Make a file that I can put in a ppt

In [36]:
import nibabel as nib
from nilearn import datasets, input_data, plotting, connectome, image
from nilearn.plotting import plot_stat_map, show
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr, zscore
import pandas as pd
import os
import json
import glob
import time
import gc
import time
from joblib import Parallel, delayed
import imageio

In [27]:
df = pd.read_csv('/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_derived/best_timestamps_networks_activation.csv')
df = df[df.network.isin(["Extended_socio-affective_default_network",
                         "Emotional_scene_and_face_processing",
                         "Theory_of_mind",
                         "Reward",
                         "Empathy",
                         "Cognitive_attention_control"
                        ])]
df

,network,movie,start,end,mean,end.1,std,method
7,Cognitive_attention_control,12yearsaslave,6585,6595.0,0.770608,6595.0,0.020846,net_median_abs_5
8,Cognitive_attention_control,backtothefuture,4014,4024.0,0.773120,4024.0,0.016471,net_median_abs_5
9,Cognitive_attention_control,littlemisssunshine,4440,4450.0,0.779985,4450.0,0.013427,net_median_abs_5
10,Cognitive_attention_control,pulpfiction,4732,4742.0,0.777150,4742.0,0.019453,net_median_abs_5
11,Cognitive_attention_control,pulpfiction,8122,8132.0,0.774127,8132.0,0.022299,net_median_abs_5
12,Cognitive_attention_control,split,3769,3779.0,0.778070,3779.0,0.016465,net_median_abs_5
13,Cognitive_attention_control,theprestige,4355,4365.0,0.766622,4365.0,0.013170,net_median_abs_5
14,Emotional_scene_and_face_processing,12yearsaslave,4855,4865.0,0.778054,4865.0,0.022897,net_median_abs_5
15,Emotional_scene_and_face_processing,pulpfiction,3742,3752.0,0.765159,3752.0,0.008119,net_median_abs_5
16,Emotional_scene_and_face_processing,pulpfiction,4992,5002.0,0.771085,5002.0,0.022342,net_median_abs_5


In [3]:
movie = "backtothefuture"
paths = glob.glob(f'/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/*/func/*{movie}_bold_blur_no_censor_ica.nii.gz')
paths

['/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/sub-75/func/sub-75_task-littlemisssunshine_bold_blur_no_censor_ica.nii.gz',
 '/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/sub-76/func/sub-76_task-littlemisssunshine_bold_blur_no_censor_ica.nii.gz',
 '/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/sub-77/func/sub-77_task-littlemisssunshine_bold_blur_no_censor_ica.nii.gz',
 '/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/sub-78/func/sub-78_task-littlemisssunshine_bold_blur_no_censor_ica.nii.gz',
 '/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/sub-79/func/sub-79_task-littlemisssunshine_bold_blur_no_censor_ica.nii.gz',
 '/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_datasets/ds002837/derivatives/sub-80/func/sub-80_task-littlemisssunshine_bold_blur_no_censor_ica.nii.gz']

In [45]:
imgs_littlemisssunshine = np.stack([nib.load(path, mmap=True).get_fdata() for path in paths], axis=0)
imgs_littlemisssunshine.shape

(6, 64, 76, 64, 5900)

In [46]:
# select time interval
start = 4014
end = 4024
select_time_interval = imgs_littlemisssunshine[:,:,:,:,start:end]
print(select_time_interval.shape)

# get the average by time and by subject
avg_img = np.mean(select_time_interval, axis=(0, 4))
print(avg_img.shape)

# create a new fmri image
fmri_img = nib.load(paths[0])
new_fmri = image.new_img_like(fmri_img, avg_img)
#nib.save(new_fmri, '/home/tamires/projects/rpp-aevans-ab/tamires/data/fmri_derived/averaged_fmri_image_test.nii.gz')


(6, 64, 76, 64, 10)
(64, 76, 64)


AttributeError: 'Nifti1Image' object has no attribute 'new_img_like'

In [ ]:
# plotting

vmin = 0.5
threshold = 2
figure_width = 8
figsize=(figure_width, 17)
plotting_config = {
    "display_mode": "ortho",
    "cut_coords": [5, -26, 21],
    "draw_cross": False,
    "vmax": 8,
    "cmap": "cold_hot"}

plot_stat_map(
    image,
    title="opaque thresholding",
    threshold=threshold,
    **plotting_config,
)

show()